# NFT Wash Trading

Here we can build a lab notebook cell by cell.

The first task is to get us both on the same page on what data we'll be using for the project, and then we can work on loading it in and working with it.

## Data import

In [1]:
##Data import code here.

## Analysis

I'll look at the paper again and determine a similar structure of analysis. Market by market, method by method.

In [2]:
##Analysis code here

## Results

In [3]:
## Hopefully some interesting results here